In [1]:
import pandas as pd
df=pd.read_csv('cleaned_house.csv')
df

,Unnamed: 0,index,selftext,num_comments,score,title
0,0,11,realize pretty vague honestly mind long simply...,0,1,Trying to source house track I heard on dubpla...
1,1,21,know long shot looking song month luck hoping ...,5,3,"Does anyone have a copy of Rene Breitbarth - ""..."
2,2,49,post dedicated finding unknown unidentifiable ...,9,4,Welcome to our weekly post for song/track IDs....
3,3,91,kind remix like best d,2,1,"Looking for a really good remix of ""Foreigner ..."
4,4,111,like minimal micro house playlist you perfect ...,0,3,Minimal House / Deep Tech Playlist (Submission...
...,...,...,...,...,...,...
4861,4861,76648,looking house track long drum break conga djem...,0,1,Looking for tracks with percussive drum breaks
4862,4862,76784,trying get year contacted label said would sho...,4,1,Need Some Help Finding a Rare Tiger &amp; Wood...
4863,4863,76797,time year yearly round best music year rule si...,13,17,r/House's 2016 Favourites
4864,4864,76800,mine in particular order hard omar s heard che...,3,11,Your favourite tracks of the year 2016?


In [2]:
df2=df.dropna(subset=['title'])[['title','selftext']]
df2 #leave it to anaylze later

,title,selftext
0,Trying to source house track I heard on dubpla...,realize pretty vague honestly mind long simply...
1,"Does anyone have a copy of Rene Breitbarth - ""...",know long shot looking song month luck hoping ...
2,Welcome to our weekly post for song/track IDs....,post dedicated finding unknown unidentifiable ...
3,"Looking for a really good remix of ""Foreigner ...",kind remix like best d
4,Minimal House / Deep Tech Playlist (Submission...,like minimal micro house playlist you perfect ...
...,...,...
4861,Looking for tracks with percussive drum breaks,looking house track long drum break conga djem...
4862,Need Some Help Finding a Rare Tiger &amp; Wood...,trying get year contacted label said would sho...
4863,r/House's 2016 Favourites,time year yearly round best music year rule si...
4864,Your favourite tracks of the year 2016?,mine in particular order hard omar s heard che...


In [3]:
#Take interesting columns
df1=df[['selftext','num_comments','score','title']]
#drop those with text=NA
df1=df1.dropna(subset=['selftext'])
#drop meaningless text
df1=df1[(df1.selftext!='[removed]') & (df1.selftext!='[deleted]')].reset_index()
#drop duplicate
duplicate = df1.duplicated()
print('There are', duplicate.sum(), 'duplicated values.')
df1=df1.drop_duplicates()
len(df1)

There are 0 duplicated values.


4287

In [4]:
Markov=df1['selftext']

### Convert to lowercase

In [5]:
df1['selftext']=df1['selftext'].str.lower()
df1['selftext']

0       realize pretty vague honestly mind long simply...
1       know long shot looking song month luck hoping ...
2       post dedicated finding unknown unidentifiable ...
3                                  kind remix like best d
4       like minimal micro house playlist you perfect ...
                              ...                        
4282    looking house track long drum break conga djem...
4283    trying get year contacted label said would sho...
4284    time year yearly round best music year rule si...
4285    mine in particular order hard omar s heard che...
4286    hey guy first post reddit please bare making m...
Name: selftext, Length: 4287, dtype: object

### Remove emoji

In [6]:
def remove_emoji(text):
    text = text.encode('ascii', 'ignore').decode()
    return text

In [7]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_emoji(x))
df1['selftext'][0]

'realize pretty vague honestly mind long simply need track house tune heard urban boogie station dubplate fm undead disco set help locating naming track appreciated'

### Remove url, punctuation, and special characters

In [8]:
#Using regular expression
import re
def remove_other(x):
    x = re.sub("\$"," ", x) #remove $
    x = re.sub("https*\S+", " ", x) #remove url
    x = re.sub("\'\w+", '', x) #remove i'm,we're,let's after the '
    x = re.sub("[0-9]+", '', x) #remove numbers
    return x

In [9]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_other(x))
df1['selftext'][0]

'realize pretty vague honestly mind long simply need track house tune heard urban boogie station dubplate fm undead disco set help locating naming track appreciated'

### Remove stopwords

In [10]:
import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords") #uncomment it when run it for the first time
stop_words = set(stopwords.words("english"))
print(stop_words) #all preloaded stopwords

{'further', 'so', 'against', 'd', 'into', 'of', 'to', 'some', 'that', 'we', 'once', "shan't", 'than', 'but', 'no', 'can', "weren't", 'during', 't', 'wasn', 'do', 've', 'when', 'very', 'where', "isn't", 'there', 'what', 'his', "wouldn't", 'is', 'ourselves', 'aren', 'was', 'who', 'her', 'up', 'on', 'a', 'for', 'from', 'down', 'won', 'how', 'hasn', 'did', 'shouldn', 'themselves', 'me', 'has', 'herself', 'isn', 'doing', "don't", 'didn', "hadn't", "you've", 'just', 'more', 'yourselves', 'those', 'theirs', 'had', 'you', 'in', 'such', "that'll", 'if', 'below', "it's", 'out', 'himself', 'above', "you're", "hasn't", 'she', 'whom', 'o', "aren't", 'these', 'ma', 'having', 'being', 'other', 'he', 'while', 'yourself', 'both', 'them', 'or', 'itself', "haven't", 'at', 'are', 'until', 'hers', 'off', 'll', 'm', 'their', 'wouldn', 'this', 'between', "mightn't", 'any', 'y', 'myself', 'should', 'shan', 'which', 'yours', 'your', 'mustn', 'mightn', 'why', 'will', 'after', "should've", "wasn't", 'not', 'few'

In [11]:
def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [12]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_stopwords(x))
df1['selftext'][0]

'realize pretty vague honestly mind long simply need track house tune heard urban boogie station dubplate fm undead disco set help locating naming track appreciated'

In [13]:
spec_chars = ["!",'“','"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df1['selftext'] = df1['selftext'].str.replace(char, ' ')
    df1['selftext'] = df1['selftext'].str.split().str.join(" ")
df1['selftext']

0       realize pretty vague honestly mind long simply...
1       know long shot looking song month luck hoping ...
2       post dedicated finding unknown unidentifiable ...
3                                    kind remix like best
4       like minimal micro house playlist perfect road...
                              ...                        
4282    looking house track long drum break conga djem...
4283    trying get year contacted label said would sho...
4284    time year yearly round best music year rule si...
4285    mine particular order hard omar heard chew sin...
4286    hey guy first post reddit please bare making m...
Name: selftext, Length: 4287, dtype: object

### Check the resulting text to see whatelse we need to clean

In [14]:
df1['selftext'][4]

'like minimal micro house playlist perfect road trip focusing relaxing living best life sure save follow like vibe amp xb'

### Example of Lemmatization(change rules to rule)

In [15]:
import nltk
#nltk.download('wordnet') #uncommented for 1st time running
from nltk.stem import WordNetLemmatizer

words=df1['selftext'][4].split()
lemmatizer = WordNetLemmatizer()
for word in words:
    print(word + " ---> " + lemmatizer.lemmatize(word))

like ---> like
minimal ---> minimal
micro ---> micro
house ---> house
playlist ---> playlist
perfect ---> perfect
road ---> road
trip ---> trip
focusing ---> focusing
relaxing ---> relaxing
living ---> living
best ---> best
life ---> life
sure ---> sure
save ---> save
follow ---> follow
like ---> like
vibe ---> vibe
amp ---> amp
xb ---> xb


In [16]:
def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    nwords=[]
    for word in words:
        word=lemmatizer.lemmatize(word)
        nwords.append(word)
    return ' '.join(nwords)

In [17]:
df1['selftext']= df1['selftext'].str.split().apply(lambda x: lemmatize(x))
df1['selftext'][4]

'like minimal micro house playlist perfect road trip focusing relaxing living best life sure save follow like vibe amp xb'

### Split words

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
vec=CountVectorizer()
counts=vec.fit_transform(df1['selftext'])
counts=counts.toarray()
count_df=pd.DataFrame(counts,columns=vec.get_feature_names())
big_df=pd.concat((df1,count_df),axis=1)
big_df.head()

MemoryError: Unable to allocate 435. MiB for an array with shape (13307, 4287) and data type int64

In [ ]:
# List of all words across rows
import itertools
import collections
def count_words(text):
    all_words = list(itertools.chain(*text.str.split()))
    counts = collections.Counter(all_words)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])

    return counts_df
count_words(df1['selftext'])

In [ ]:
stop_words.update({'would','k','im','could','also','amp','much','one','like','get','since'})
df1['selftext']=df1['selftext'].apply(lambda x: remove_stopwords(x))
count_df=count_words(df1['selftext'])
count_df.head(30)

In [ ]:
import plotly.express as px
fig = px.bar(count_df.head(10), x='words', y='count', title="Top 10 words in Financial Independence")
fig.show()

In [ ]:
# Libraries
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a list of word
text=' '.join(count_df.words)

# Create the wordcloud object
wordcloud = WordCloud(width=1500, height=1000, margin=0,background_color='white').generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

### Two grams Frequency

In [ ]:
from nltk.util import ngrams
def count_2gram(text):
    all_words = list(itertools.chain(*text.str.split()))
    esBigrams = ngrams(all_words, 2)
    counts = collections.Counter(esBigrams)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])
    dictionary2 = [' '.join(tup) for tup in counts_df.words]
    counts_df.words=dictionary2

    return counts_df
counts_df2=count_2gram(df1['selftext'])
counts_df2.head(20)

In [ ]:
fig = px.bar(counts_df2.head(10), x='words', y='count', title="Top 10 2-grams in Financial Independence")
fig.show()

In [ ]:
words_dict = dict(counts_df2.values)

wordCloud = WordCloud(max_words=200, height=1000, width=1500)
wordCloud.generate_from_frequencies(words_dict)
plt.title('Most frequently occurring bigrams connected by same colour and font size')
plt.imshow(wordCloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Try Markov Chain to generate synthetic text

In [ ]:
import random
def markov_text(s, n, seed,length):
    '''
    Generates synthetic text according to an n-th order Markov model
    Parameter s: input string of real text
    Parameter n: integer n, the order of the model
    Parameter seed: the initial string that gets the Markov model started.
    Parameter length: integer, the size of the text to generate. default=100
    '''
    counts = count_ngrams(s, n+1)
    fake=seed
    for i in range(length):
        previous = fake[(-n):]
        # filter dict to keep only matching grams
        sub = {}
        for key in counts:
            if key[:-1] == previous: 
                sub[key] = counts[key]
    # convert to lists for use with random.choices
        choices = list(sub.keys())
        weights = [sub[key] for key in choices]
        new_gram = random.choices(choices, weights)[0]
        new_char = new_gram[-1]
        fake+=new_char
    return fake 

In [ ]:
def count_ngrams(s,n=1):
    '''
    Counts the number of times each n-gram occurs in a string 's'
    Parameter s: input string
    Parameter n: input integer n to specify 'n-gram' that need to be count
    '''
    D={}
    #loop through s, untill there are not enough characters left forming n-gram
    for i in range(len(s)-(n-1)): 
        D[s[i:i+n]]=0 # initialize values to avoid key errors
    for i in range(len(s)-(n-1)):
        D[s[i:i+n]]+=1 #the value of each n-gram is their counts in s
    return D
    
#test the output
count_ngrams("tortoise", n = 2)

In [ ]:
import itertools
all_words = list(itertools.chain(*Markov.str.split()))
s=' '.join(all_words)

In [ ]:
markov_text(s, n = 8, length = 400, seed = "roth ira")